In [ ]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [ ]:
//1. Local version: clone this repository, run `sbt publishLocal`
//2. Remote version: nothing to do (but you depend on what versions are available)
import $ivy.`com.github.dzufferey::scadla:0.1.2-SNAPSHOT`

In [ ]:
import scadla.{Cube,Sphere,Minkowski,Cylinder,Union,Difference}
import scadla.InlineOps.Ops
import scadla.EverythingIsIn.{millimeters, degrees}
import scadla.backends.OpenSCAD // for rendering (getting a mesh)
import scadla.backends.almond.Viewer // to show the mesh in jupyter/almond

In [ ]:
// exmaple from the README
val c = Cube(1, 1, 1)
val s = Sphere(1.5)
val obj = (c + c.move(-0.5, -0.5, 0)) * s
val mesh = OpenSCAD(obj) // Solid -> Polyhedron
Viewer(mesh)

In [ ]:
// a more complex example
import squants.space.{Length, Angle}
import scadla.utils.{RoundedCubeH, Trig, Trapezoid}

// dimensions
val baseWidth: Length = 95
val baseLength: Length = 150
val baseDepth: Length = 5

val overallWidth: Length = 200
val overallLength = baseLength
val cornerRadius: Length  = 10
val wall: Length = 2
val wallStraight: Length = 40
val wallSlopped: Length = 10
val wallAngle: Angle = 45

val grooveWidth: Length = 3
val grooveDepth: Length = 2
val nbrGrooveH = 12
val nbrGrooveV = 10

def hat(x: Length, y: Length, z: Length, r: Length, a: Angle) = {
    val d = r*2
    val smaller = z * Trig.sin(a)
    Minkowski(
        Trapezoid((x-d-smaller,x-d,0),(y-d-smaller,y-d,0),z-1),
        Cylinder(r,1)
    ).move(r,r,0)
}

val base = Union(
    Cube(baseWidth, baseLength, baseDepth).move((overallWidth-baseWidth)/2,(overallLength-baseLength)/2,-baseDepth),
    RoundedCubeH(overallWidth, overallLength, wallStraight, cornerRadius),
    hat(overallWidth, overallLength, wallSlopped, cornerRadius, wallAngle).moveZ(wallStraight)
)
val sampleGrooveH = Cube(grooveWidth, overallLength - wall*2 - 10, grooveDepth)
val stepH = (overallWidth - wall*2 - 10) / nbrGrooveH
val groovesH = for (i <- 1 until nbrGrooveH) yield sampleGrooveH.move(wall + 5 + stepH*i - grooveWidth/2, wall+5, wall)
val sampleGrooveV = Cube(overallWidth - wall*2 - 10, grooveWidth, grooveDepth)
val stepV = (overallLength - wall*2 - 10) / nbrGrooveV
val groovesV = for (i <- 1 until nbrGrooveV) yield sampleGrooveV.move(wall + 5, wall + 5 + stepV*i - grooveWidth/2,wall)
val shell = Difference(
    base,
    RoundedCubeH(overallWidth - wall*2, overallLength - wall*2, wallStraight-wall-grooveDepth, cornerRadius-wall).move(wall,wall,wall+grooveDepth),
    hat(overallWidth - wall*2, overallLength - wall*2, wallSlopped, cornerRadius-wall, wallAngle).move(wall,wall,wallStraight),
    Union(groovesH: _*),
    Union(groovesV: _*)
)
val mesh = OpenSCAD(shell)
Viewer(mesh)

In [ ]:
// save the object as an STL file
scadla.backends.stl.Printer.storeBinary(mesh, "milling_basket.stl")